In [1]:
!pip install -q gdown

# kalian download semua file dari gdrive dibawah lalu buat folder (ex. midterm_folder)
!gdown --folder 1JvI5xhPfN3VmjpWYZk9fCHG41xG697um -O midterm_folder

Retrieving folder contents
Processing file 1ALveidCmKzk4p_liQ7b663BXL0PcTaFN test_transaction.csv
Processing file 1Ujh37a1kTarNf3dSOSrj-hKDswULebYA train_transaction.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1ALveidCmKzk4p_liQ7b663BXL0PcTaFN

but Gdown can't. Please check connections and permissions.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import polars as pl

BASE_PATH = "/content/drive/MyDrive/ML & DL"

train_transaction = pl.read_csv(f"/content/drive/MyDrive/ML & DL/train_transaction.csv")
test_transaction  = pl.read_csv(f"/content/drive/MyDrive/ML & DL/test_transaction.csv")


print(train_transaction.shape) #590540 row dan 393 feature + 1 target
print(test_transaction.shape) #506691 row dan 393 feature

(590540, 394)
(506691, 393)


In [5]:
train_transaction.head() #isFraud adalah target featurenya

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,"""W""",13926,null,150.0,"""discover""",142.0,"""credit""",315.0,87.0,19.0,null,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,"""W""",2755,404.0,150.0,"""mastercard""",102.0,"""credit""",325.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,"""W""",4663,490.0,150.0,"""visa""",166.0,"""debit""",330.0,87.0,287.0,null,"""outlook.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,"""W""",18132,567.0,150.0,"""mastercard""",117.0,"""debit""",476.0,87.0,null,null,"""yahoo.com""",null,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,"""H""",4497,514.0,150.0,"""mastercard""",102.0,"""credit""",420.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test_transaction.head() #uji data baru, setelah pembuatan model machine learningnya

TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3663549,18403224,31.95,"""W""",10409,111.0,150.0,"""visa""",226.0,"""debit""",170.0,87.0,1.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,null,null,…,0.0,0.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663550,18403263,49.0,"""W""",4272,111.0,150.0,"""visa""",226.0,"""debit""",299.0,87.0,4.0,null,"""aol.com""",null,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,null,null,…,0.0,0.0,1.0,0.0,280.0,77.0,0.0,280.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663551,18403310,171.0,"""W""",4476,574.0,150.0,"""visa""",226.0,"""debit""",472.0,87.0,2635.0,null,"""hotmail.com""",null,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,null,null,…,0.0,0.0,1.0,0.0,1321.0,0.0,0.0,1058.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663552,18403310,284.95,"""W""",10989,360.0,150.0,"""visa""",166.0,"""debit""",205.0,87.0,17.0,null,"""gmail.com""",null,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663553,18403317,67.95,"""W""",18018,452.0,150.0,"""mastercard""",117.0,"""debit""",264.0,87.0,6.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,null,null,…,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [7]:
import gc

# Gunakan BASE_PATH yang sama dengan kodemu sebelumnya
# Pastikan file identity ada di folder "/content/drive/MyDrive/ML & DL"
print("⏳ Sedang memuat dan menggabungkan data Identitas...")

try:
    # 1. Load Data Identity
    train_identity = pl.read_csv(f"/content/drive/MyDrive/ML & DL/train_transaction.csv")
    test_identity  = pl.read_csv(f"/content/drive/MyDrive/ML & DL/test_transaction.csv")

    # 2. Gabungkan (Merge) berdasarkan 'TransactionID'
    # Kita pakai 'left join' agar semua data transaksi tetap ada
    # Hasilnya disimpan di variabel baru 'df_train' dan 'df_test'
    df_train = train_transaction.join(train_identity, on='TransactionID', how='left')
    df_test  = test_transaction.join(test_identity, on='TransactionID', how='left')

    # 3. Hapus variabel lama untuk hemat RAM (Penting!)
    del train_transaction, test_transaction, train_identity, test_identity
    gc.collect()

    print("✅ BERHASIL MERGE!")
    print(f"Ukuran Data Train Baru: {df_train.shape} (Harusnya kolom bertambah jadi 434)")
    print(f"Ukuran Data Test Baru : {df_test.shape}")

except Exception as e:
    print("❌ Terjadi Error:", e)
    print("Pastikan file 'train_identity.csv' dan 'test_identity.csv' ada di folder Drive kamu.")

⏳ Sedang memuat dan menggabungkan data Identitas...
✅ BERHASIL MERGE!
Ukuran Data Train Baru: (590540, 787) (Harusnya kolom bertambah jadi 434)
Ukuran Data Test Baru : (506691, 785)


In [8]:
# Kita ambil sampel data agar RAM tidak jebol saat konversi nanti
# 100.000 baris sudah sangat cukup untuk membuktikan model ML bekerja
print("✂️ Sedang melakukan Sampling Data...")

df_train = df_train.sample(n=100000, seed=42)
df_test  = df_test.sample(n=50000, seed=42)

print("✅ Sampling Selesai.")
print(f"Ukuran Data Train Baru: {df_train.shape}")
print(f"Ukuran Data Test Baru : {df_test.shape}")

✂️ Sedang melakukan Sampling Data...
✅ Sampling Selesai.
Ukuran Data Train Baru: (100000, 787)
Ukuran Data Test Baru : (50000, 785)


In [9]:
import numpy as np

print("🚀 Mulai Preprocessing Data...")

# 1. Pisahkan Target (Jawaban) dan Fitur
# Kita buang 'TransactionID' dan 'TransactionDT' karena cuma penanda urutan (bukan ciri fraud)
y_train = df_train['isFraud']
X_train = df_train.drop(['isFraud', 'TransactionID', 'TransactionDT'])
X_test  = df_test.drop(['TransactionID', 'TransactionDT'])

# 2. Penyelarasan Kolom (PENTING!)
# Tadi jumlah kolom beda (787 vs 785). Kita ambil irisan kolom yang ada di KEDUA tabel saja.
# Supaya Train dan Test punya fitur yang 100% sama.
common_cols = [col for col in X_train.columns if col in X_test.columns]
X_train = X_train.select(common_cols)
X_test  = X_test.select(common_cols)

print(f"✅ Kolom disamakan. Jumlah fitur sekarang: {len(common_cols)}")

# 3. Encoding (Ubah Teks jadi Angka)
# Cari kolom yang tipe datanya String/Utf8
cat_cols = [col for col in X_train.columns if X_train[col].dtype == pl.Utf8]
print(f"🔄 Mengubah {len(cat_cols)} kolom teks menjadi angka...")

for col in cat_cols:
    # Ubah jadi kategori lalu ke kode angka (0, 1, 2...)
    X_train = X_train.with_columns(pl.col(col).cast(pl.Categorical).to_physical().fill_null(-1))
    X_test  = X_test.with_columns(pl.col(col).cast(pl.Categorical).to_physical().fill_null(-1))

# 4. Isi Data Kosong (Null) dengan -1
X_train = X_train.fill_null(-1)
X_test  = X_test.fill_null(-1)

# 5. Konversi ke Numpy Array (Float32 biar Hemat RAM)
print("📦 Konversi ke Numpy Array...")
X_train_np = X_train.to_numpy().astype(np.float32)
y_train_np = y_train.to_numpy().astype(np.float32)
X_test_np  = X_test.to_numpy().astype(np.float32)

# Bersihkan RAM (Hapus dataframe Polars)
del df_train, df_test, X_train, y_train, X_test, common_cols
gc.collect()

print("✅ Preprocessing Selesai! Data siap Training.")
print(f"Dimensi Akhir: {X_train_np.shape}")

🚀 Mulai Preprocessing Data...
✅ Kolom disamakan. Jumlah fitur sekarang: 783
🔄 Mengubah 28 kolom teks menjadi angka...
📦 Konversi ke Numpy Array...
✅ Preprocessing Selesai! Data siap Training.
Dimensi Akhir: (100000, 783)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

print("🤖 Mulai Training Random Forest...")

# 1. Bagi Data Train jadi Dua: Training (80%) & Validasi (20%)
# Ini wajib biar kita tahu seberapa pinter modelnya (karena data Test asli gapunya jawaban)
X_fit, X_val, y_fit, y_val = train_test_split(X_train_np, y_train_np, test_size=0.2, random_state=42)

# 2. Inisialisasi Model Random Forest
# Kita set parameter biar tidak makan RAM banyak tapi tetap akurat
rf_model = RandomForestClassifier(
    n_estimators=100,        # Jumlah pohon (Standar)
    max_depth=15,            # Kedalaman pohon dibatasi biar gak Overfitting & Hemat RAM
    class_weight='balanced', # Menangani Imbalance (PENTING Sesuai Soal)
    n_jobs=-1,               # Pakai semua core CPU biar ngebut
    random_state=42,
    verbose=1
)

# 3. Latih Model (Fitting)
rf_model.fit(X_fit, y_fit)

# 4. Evaluasi Hasil
print("\n📊 Evaluasi Model (Validation Set):")
y_pred_val_prob = rf_model.predict_proba(X_val)[:, 1] # Ambil probabilitas Fraud (1)
auc_score = roc_auc_score(y_val, y_pred_val_prob)

print(f"✅ AUC Score: {auc_score:.4f}")
print("(Nilai di atas 0.80 artinya Bagus, di atas 0.90 artinya Sangat Bagus)")

# 5. Prediksi Data Test Asli (Untuk Submission)
print("\n🔮 Memprediksi Data Test Asli...")
y_test_pred = rf_model.predict_proba(X_test_np)[:, 1]

print("Contoh 5 Prediksi Pertama pada Data Test:")
print(y_test_pred[:5])

print("\n🎉 TUGAS MACHINE LEARNING SELESAI!")

🤖 Mulai Training Random Forest...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   46.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.



📊 Evaluasi Model (Validation Set):


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


✅ AUC Score: 0.8755
(Nilai di atas 0.80 artinya Bagus, di atas 0.90 artinya Sangat Bagus)

🔮 Memprediksi Data Test Asli...


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s


Contoh 5 Prediksi Pertama pada Data Test:
[0.10380026 0.44100132 0.03104218 0.25936326 0.16884386]

🎉 TUGAS MACHINE LEARNING SELESAI!


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.1s finished
